In [35]:
import helpers as hp
import nltk
import csv
from nltk.corpus import wordnet as wn
import preprocess 
from preprocess import pos_tag_db, lemmatize_db
import csv
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from re import search
# Get a list of available databases
dbs = hp.getDatabases()

In [36]:
#creating a list of ingredients
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import sys
url = "https://world.openfoodfacts.org/ingredients.json"
page = urlopen(url)
html = page.read()
soup = BeautifulSoup(html) 
u = soup.decode('utf-8')
u = u.split('"name":')
#creating a csv file with ingredients
localFile = open('ingredients.csv', 'w')
#creating a list of ingredients
ingredients = []
for line in u:
    ingredients.append(line.split(',')[0])
del ingredients[0]
for ingredient in ingredients:
    localFile.write(ingredient)
localFile.close()


In [37]:
#we have 10000 ingredients in this vegetarian dataset
len(ingredients)

10000

In [38]:
for element in ingredients:
    #remove any string that contains digits
    if any(map(str.isdigit, element)):
        ingredients.remove(element)
        
        
   # if "E1" or "E2" or "E3" or "E4" or "E9" in element:
    #    ingredients.remove(element)
if "ingredient" in ingredients:
    ingredients.remove("ingredient")
 

In [39]:
ingredients

['"Salt"',
 '"Sugar"',
 '"Water"',
 '"Flavouring"',
 '"Oil and fat"',
 '"Vegetable oil and fat"',
 '"Dairy"',
 '"Vegetable"',
 '"Cereal"',
 '"Natural flavouring"',
 '"Vegetable oil"',
 '"Flour"',
 '"Root vegetable"',
 '"Wheat"',
 '"Fruit"',
 '"Starch"',
 '"Spice"',
 '"Cereal flour"',
 '"Glucose"',
 '"Milk"',
 '"Wheat flour"',
 '"Seed"',
 '"Onion"',
 '"Colour"',
 '"Garlic"',
 '"Vitamins"',
 '"Preservative"',
 '"Minerals"',
 '"Cocoa"',
 '"Corn starch"',
 '"Soya lecithin"',
 '"Egg"',
 '"Yeast"',
 '"Pepper"',
 '"Emulsifier"',
 '"Modified starch"',
 '"Herb"',
 '"Dextrose"',
 '"Plant"',
 '"Corn syrup"',
 '"Vinegar"',
 '"Rapeseed oil"',
 '"Nut"',
 '"Thiamin"',
 '"Sunflower oil"',
 '"Iron"',
 '"Palm oil and fat"',
 '"Folate"',
 '"Folic acid"',
 '"Tomato"',
 '"Sea salt"',
 '"Milk powder"',
 '"Ferment"',
 '"Cream"',
 '"Enzyme"',
 '"Palm oil"',
 '"Artificial flavouring"',
 '"Thiamin mononitrate"',
 '"Protein"',
 '"Cocoa butter"',
 '"Fruit juice"',
 '"Citrus fruit"',
 '"Tree nut"',
 '"Microbial cu

In [40]:
ingredients_str = str(ingredients)

## First database : 'VegetarianRecipes.exl'

In [41]:
#data structure of clean_db is a list of strings
clean_db = hp.cleanFile(dbs[0])
  
#merging the cleaned sentenes 
vegetarian_recipes = []
for sentence in clean_db:
    vegetarian_recipes.append(sentence)
#split into separate recipes
split_vegetarian_recipes = str(vegetarian_recipes).split('EACH')
del split_vegetarian_recipes[0]

In [42]:
#writing the recipes to a csv file
with open('recipes_veg.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHOD", "NOTE", "CONTAINS (NOUNS)", "INGREDIENTS"])
    #loop through each recipes 
    allmethod = []
    for recipe in split_vegetarian_recipes:
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        contains = []
        for sent in re.split(',|\n', recipe):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
        #merge method (as a list) into one single string
        method_str = ""
        for text in method:
            method_str += str(text) + " "
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        method_str = tokenizer.tokenize(method_str)
        #lemmatize and POS tag the method string
        #PoS-Tag, lemmatize and format
        postagged = pos_tag_db(method_str)
        lemmatized = lemmatize_db(postagged, [".", "X"])
        formatted = hp.formatFile(lemmatized)
        method_tagged = formatted
        onelist = []
        for pair in method_tagged:
            onelist.extend(pair)
        for (word, tag) in onelist:
            if (tag == 'NOUN'):
                contains.append(word)
        #from contains we can filter for features like ingredients and tools
        ingre = []
        for contain in contains:
            if search(contain, str(ingredients)) and len(contain) > 1:
                ingre.append(contain)
        #write row for each recipe
        #the ingredients are turned into a set, so there is no repetition of ingredients.
        writer.writerow([portion, temperature, onelist, note, contains, set(ingre)])   

## Second database: 'data/EthnicRecipes.exl' Problem?

In [43]:
#where's the method??

clean_db = hp.cleanFile(dbs[1])
#merging the cleaned sentenes 
ethnic_recipes = []
for sentence in clean_db:
    ethnic_recipes.append(sentence)
#split the recipes into lists
split_ethnic_recipes = str(ethnic_recipes).split('COOK')

## Third database: data/ArmedForcesRecipes.exl'

In [51]:
clean_db = hp.cleanFile(dbs[2])
armedforces_recipes = []
for sentence in clean_db:
    armedforces_recipes.append(sentence)
#split the recipes into lists
split_armedforces_recipes = str(armedforces_recipes).split('EACH')
del split_armedforces_recipes[0]

In [ ]:
#writing the recipes to a csv file
with open('recipes_armed.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHOD", "NOTE", "CONTAINS (NOUNS)", "INGREDIENTS"])
    #loop through each recipes 
    allmethod = []
    for recipe in split_armedforces_recipes:
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        contains = []
        for sent in re.split(',|\n', recipe):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
        #merge method (as a list) into one single string
        method_str = ""
        for text in method:
            method_str += str(text) + " "
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        method_str = tokenizer.tokenize(method_str)
        #lemmatize and POS tag the method string
        #PoS-Tag, lemmatize and format
        postagged = pos_tag_db(method_str)
        lemmatized = lemmatize_db(postagged, [".", "X"])
        formatted = hp.formatFile(lemmatized)
        method_tagged = formatted
        onelist = []
        for pair in method_tagged:
            onelist.extend(pair)
        for (word, tag) in onelist:
            if (tag == 'NOUN'):
                contains.append(word)
        #from contains we can filter for features like ingredients and tools
        ingre = []
        for contain in contains:
            if search(contain, str(ingredients)) and len(contain) > 1:
                ingre.append(contain)
        #write row for each recipe
        #the ingredients are turned into a set, so there is no repetition of ingredients.
        writer.writerow([portion, temperature, onelist, note, contains, set(ingre)]) 

## Fourth database: 'data/CommonRecipes.exl'

In [47]:
clean_db = hp.cleanFile(dbs[3])
common_recipes = []
for sentence in clean_db:
    common_recipes.append(sentence)
#split the recipes into lists
split_common_recipes = str(common_recipes).split('EACH')

In [50]:
#writing the recipes to a csv file
with open('recipes_common.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHOD", "NOTE", "CONTAINS (NOUNS)", "INGREDIENTS"])
    #loop through each recipes 
    allmethod = []
    for recipe in split_common_recipes:
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        contains = []
        for sent in re.split(',|\n', recipe):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
        #merge method (as a list) into one single string
        method_str = ""
        for text in method:
            method_str += str(text) + " "
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        method_str = tokenizer.tokenize(method_str)
        #lemmatize and POS tag the method string
        #PoS-Tag, lemmatize and format
        postagged = pos_tag_db(method_str)
        lemmatized = lemmatize_db(postagged, [".", "X"])
        formatted = hp.formatFile(lemmatized)
        method_tagged = formatted
        onelist = []
        for pair in method_tagged:
            onelist.extend(pair)
        for (word, tag) in onelist:
            if (tag == 'NOUN'):
                contains.append(word)
        #from contains we can filter for features like ingredients and tools
        ingre = []
        for contain in contains:
            if search(contain, str(ingredients)) and len(contain) > 1:
                ingre.append(contain)
        #write row for each recipe
        #the ingredients are turned into a set, so there is no repetition of ingredients.
        writer.writerow([portion, temperature, onelist, note, contains, set(ingre)]) 